In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()


25/03/05 02:54:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
df = spark.sql("SELECT count(*) FROM demo.nyc.taxis_100M_50")
df.show()

+---------+
| count(1)|
+---------+
|100000000|
+---------+



In [9]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Initialize Spark session with optimized configurations
spark = SparkSession.builder \
    .appName("Optimized Iceberg Update") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

# Load the Iceberg table
df = spark.table("demo.nyc.taxis_100M_50")

# Get total row count
total_rows = df.count()
print(f"Total rows in table: {total_rows}")

# Get update percentage from user
update_percentage = float(input("Enter update percentage (e.g., 1 for 1%): ").strip()) / 100
num_rows = max(1, int(total_rows * update_percentage))

# Define batch size
batch_size = 10000  # Optimized for large datasets
num_batches = (num_rows // batch_size) + (1 if num_rows % batch_size else 0)

print(f"Updating {num_rows} rows (~{update_percentage*100}%) in {num_batches} batches...")

start_time = time.time()

for batch_num in range(num_batches):
    print(f"Processing batch {batch_num + 1}/{num_batches}...")

    # Sample records to update
    sampled_df = df.select("extra_col_0").sample(False, batch_size / total_rows).limit(batch_size)

    # Create a temporary view for sampled records
    sampled_df.createOrReplaceTempView("updates")

    # Perform the update using Iceberg MERGE INTO SQL
    spark.sql("""
        MERGE INTO demo.nyc.taxis_100M_50 AS target
        USING updates AS source
        ON target.extra_col_0 = source.extra_col_0
        WHEN MATCHED THEN
        UPDATE SET target.extra_col_1 = target.extra_col_1 + 10
    """)

    print(f"Batch {batch_num + 1}/{num_batches} updated.")

end_time = time.time() - start_time
print(f"\nTotal update time for {num_batches} batches: {end_time:.2f} sec")


25/03/05 02:54:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Total rows in table: 100000000


Enter update percentage (e.g., 1 for 1%):  1


Updating 1000000 rows (~1.0%) in 100 batches...
Processing batch 1/100...


25/03/05 02:59:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/03/05 02:59:15 WARN RowBasedKeyValueBatch: Calling spill() on

Py4JJavaError: An error occurred while calling o39.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 71 in stage 260.0 failed 1 times, most recent failure: Lost task 71.0 in stage 260.0 (TID 6219) (5e0ed98abed9 executor driver): java.util.concurrent.CompletionException: software.amazon.awssdk.core.exception.SdkClientException: Unable to execute HTTP request: Connection reset by peer (Write failed)
	at java.base/java.util.concurrent.CompletableFuture.encodeThrowable(CompletableFuture.java:314)
	at java.base/java.util.concurrent.CompletableFuture.completeThrowable(CompletableFuture.java:319)
	at java.base/java.util.concurrent.CompletableFuture$AsyncSupply.run(CompletableFuture.java:1702)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: software.amazon.awssdk.co

In [5]:
df = spark.sql("SELECT snapshot_id, made_current_at FROM demo.nyc.taxis_10M_50COLUMNS.history ORDER BY made_current_at DESC")

df.show()

+-------------------+--------------------+
|        snapshot_id|     made_current_at|
+-------------------+--------------------+
| 105668051919482591|2025-03-04 10:00:...|
|6814201661817395182|2025-03-04 09:42:...|
|6521404266517949867|2025-03-04 09:39:...|
|7604542288563918334|2025-03-04 09:36:...|
| 552092269728169030|2025-03-04 09:33:...|
|2209669498964280618|2025-03-04 09:30:...|
|7674319906980595297|2025-03-04 09:27:...|
|5468151910761078557|2025-03-04 09:23:...|
|7947044984117739091|2025-03-04 09:20:...|
|5673072808774441768|2025-03-04 09:17:...|
|1470626163957892397|2025-03-04 09:09:...|
|7160248633578306637|2025-03-04 09:06:...|
|5174434347034670474|2025-03-04 08:47:...|
|1794110359935204866|2025-03-04 08:44:...|
|6937317397142763324|2025-03-04 08:41:...|
|3089841895765010104|2025-03-04 08:38:...|
|7127818708494625786|2025-03-04 08:34:...|
|2707916599315632908|2025-03-04 08:30:...|
| 497229154627925394|2025-03-04 08:25:...|
|7776288292023736999|2025-03-04 08:13:...|
+----------

In [8]:
df=spark.sql("SELECT * FROM demo.nyc.taxis_10M_50COLUMNS.snapshots ORDER BY committed_at desc")
df.show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-03-04 10:00:...| 105668051919482591|6814201661817395182|overwrite|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:42:...|6814201661817395182|6521404266517949867|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:39:...|6521404266517949867|7604542288563918334|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:36:...|7604542288563918334| 552092269728169030|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:33:...| 552092269728169030|2209669498964280618|   append|s3://warehouse/ny...|{spark.app.id -> ...|
|2025-03-04 09:30:...|2209669498964280618|7674319906980595297|   append|s3://war